# 08 - Explainability (SHAP & LIME)

Goal: provide transparent explanations for model decisions, aligning with ethical and regulatory expectations.

Tools:
- SHAP (TreeExplainer for tree-based models)
- LIME (local explanations for individual predictions)

We use models trained in earlier notebooks; no new training happens here.

In [ ]:
import shap
import lime
from lime import lime_tabular
import pandas as pd
import joblib
from pathlib import Path

PROCESSED_DIR = Path('data/processed')
X_train = pd.read_csv(PROCESSED_DIR / 'X_train_scaled.csv')
X_test = pd.read_csv(PROCESSED_DIR / 'X_test_scaled.csv')

# Placeholder: load a trained tree model (e.g., RandomForest)
# rf_model = joblib.load('models/saved_models/random_forest.joblib')
# explainer = shap.TreeExplainer(rf_model)
# shap_values = explainer.shap_values(X_test)

# For LIME
# lime_explainer = lime_tabular.LimeTabularExplainer(
#     training_data=X_train.values,
#     feature_names=X_train.columns,
#     class_names=['non-fraud', 'fraud'],
#     mode='classification'
# )
# i = 0
# lime_exp = lime_explainer.explain_instance(X_test.iloc[i].values, rf_model.predict_proba)


Discuss ethical implications, model transparency, and how explanations support compliance (e.g., GDPR).